# Check undownloaded data

In [10]:
import os
from datetime import datetime
from codes import codes

def dlcheck(path):
    filename=os.listdir(path)
    code_cap=[]
    for _file in filename:
        code_cap.append(_file[:-8])
    code_cap=set(code_cap)
    return code_cap



#Input Parameter
#code='2330'
year, month =2019,5
if month >9:
    jsonpath= str(year)+str(month)
else:
    jsonpath= str(year)+'0'+str(month)

code_cap=dlcheck(jsonpath)
#print('already downloaded:', code_cap)

ym=datetime.strptime(str(year)+'-'+str(month), "%Y-%m")

#print(fc)
undownload=[]
for code, v in codes.items():
    if v.type=="股票" and v.market=="上市":
        fc='{}_{:%Y-%m}'.format(code, ym)
        if fc not in code_cap:
            undownload.append(fc)
            #print('undownloaded:{}'.format(fc))
            
print('undownlowed count:', len(undownload))
print('[undownload file list]')

for _ in undownload:
    print(_)

undownlowed count: 0
[undownload file list]


# Complete Code include undowanload check

In [21]:
from TWstockPrice import Stock
import time
import random
import os
from requests.exceptions import ConnectionError
import json
from codes import codes
from datetime import datetime

def dlcheck(path):
    filename=os.listdir(path)
    code_cap=[]
    for _file in filename:
        code_cap.append(_file[:-8])
    code_cap=set(code_cap)
    return code_cap

year, month =2019,1

if month >9:
    jsonpath= str(year)+str(month)
else:
    jsonpath= str(year)+'0'+str(month)


if not os.path.exists(jsonpath):
    os.mkdir(jsonpath)
    
stockDict={}
stockList=[]
ym=datetime.strptime(str(year)+'-'+str(month), "%Y-%m")

undownload=[]
code_cap=dlcheck(jsonpath)
for code, v in codes.items():
    if v.type=="股票" and v.market=="上市":
        fc='{}_{:%Y-%m}'.format(code, ym)
        if fc not in code_cap:
            undownload.append(fc)
            #print('undownloaded:{}'.format(fc))         
print('undownlowed count:', len(undownload))
#print('[undownload file list]')


cycle=0
while len(undownload)!=0:
    cycle+=1
    print('downloading cycle:', cycle )
    for code,v in codes.items():
        if v.type=="股票" and v.market=="上市":
            try:
                #print(code)
                fc='{}_{:%Y-%m}'.format(code, ym)
                #print(fc)   
                if fc not in code_cap:
                    stock= Stock(code)
                    stock.fetch(year, month)
                    print('write to:', fc)

                    stockDict={}
                    if len(stock.data) == 0:
                        print('no data:{}'.format(code))
                        filename=fc+'-ng.json'
                        with open(jsonpath+'/'+filename, 'w') as f:
                            json.dump(stockDict, f)

                    else:               
                        for _data in stock.data:
                            ymd='{:%Y-%m-%d}'.format(_data.date)
                            filename=code+'_'+ymd+'.json'
                            stockDict['code'] =code           
                            content =dict(_data._asdict())
                            for k, v in content.items():
                                if k is 'date':
                                    stockDict[k] = '{:%Y-%m-%d}'.format(_data.date)
                                else:
                                    stockDict[k] = v

                            #print('write to:', filename)
                            with open(jsonpath+'/'+filename, 'w') as f:
                                json.dump(stockDict, f)

            except ConnectionError:
                code_cap=dlcheck(jsonpath)
                print('ConnectionError')
                #break
                time.sleep(30)
                continue
                
    undownload=[]
    code_cap=dlcheck(jsonpath)
    for code, v in codes.items():
        if v.type=="股票" and v.market=="上市":
            fc='{}_{:%Y-%m}'.format(code, ym)
            if fc not in code_cap:
                undownload.append(fc)
                #print('undownloaded:{}'.format(fc))
    print('undownlowed count:', len(undownload))          

print('complete!!')

undownlowed count: 858
downloading cycle: 1
write to: 1463_2019-01
write to: 1464_2019-01
write to: 1465_2019-01
write to: 1466_2019-01
write to: 1467_2019-01


KeyboardInterrupt: 

# Modify dlcheck

In [ ]:
from TWstockPrice import Stock
import time
import random
import os
from requests.exceptions import ConnectionError
import json
from codes import codes
from datetime import datetime

def dlcheck(path, ym):
    filename=os.listdir(path)
    code_cap=[]
    
    for _file in filename:
        code_cap.append(_file[:-8])
    code_cap=set(code_cap)
    undownload=[]
    
    for code, v in codes.items():
        if v.type=="股票" and v.market=="上市":
            fc='{}_{:%Y-%m}'.format(code, ym)
            if fc not in code_cap:
                undownload.append(fc)
            #print('undownloaded:{}'.format(fc))         
    return undownload

#print('[undownload file list]')

year, month =2019,1

if month >9:
    jsonpath= str(year)+str(month)
else:
    jsonpath= str(year)+'0'+str(month)


if not os.path.exists(jsonpath):
    os.mkdir(jsonpath)
    
stockDict={}
stockList=[]
ym=datetime.strptime(str(year)+'-'+str(month), "%Y-%m")

undownload=dlcheck(jsonpath, ym)
print('undownlowed count:', len(undownload))


cycle=0
while len(undownload)!=0:
    cycle+=1
    print('downloading cycle:', cycle )
    for fc in undownload:
        try:
            code = fc[:4]
            #print(fc)
            #print(code)           
            stock= Stock(code)
            stock.fetch(year, month)
            print('write to:', fc)
            stockDict={}
            if len(stock.data) == 0:
                print('no data:{}'.format(code))
                filename=fc+'-ng.json'
                with open(jsonpath+'/'+filename, 'w') as f:
                    json.dump(stockDict, f)

            else:               
                for _data in stock.data:
                    ymd='{:%Y-%m-%d}'.format(_data.date)
                    filename=code+'_'+ymd+'.json'
                    stockDict['code'] =code           
                    content =dict(_data._asdict())
                    for k, v in content.items():
                        if k is 'date':
                            stockDict[k] = '{:%Y-%m-%d}'.format(_data.date)
                        else:
                            stockDict[k] = v

                    #print('write to:', filename)
                    with open(jsonpath+'/'+filename, 'w') as f:
                        json.dump(stockDict, f)

        except ConnectionError:
            undownload=dlcheck(jsonpath)
            print('ConnectionError')
            #break
            time.sleep(30)
            continue
                
    undownload=dlcheck(jsonpath, ym)
    print('undownlowed count:', len(undownload))           

print('complete!!')

undownlowed count: 852
downloading cycle: 1
write to: 1470_2019-01
write to: 1471_2019-01
write to: 1472_2019-01
write to: 1473_2019-01
write to: 1474_2019-01
write to: 1475_2019-01
write to: 1476_2019-01
